In [2]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import os 

In [22]:
data_path = '/home/sonic/Desktop/104_data1/jupyter/sungjoo/VF/PseudoEdgeNet/PseudoEdgeNet_Nuclei_Segmentation_only_with_Point_Annotations-master/data/lateral/Dataset'
mask_path ='/home/sonic/Desktop/104_data1/jupyter/sungjoo/VF/PseudoEdgeNet/PseudoEdgeNet_Nuclei_Segmentation_only_with_Point_Annotations-master/data/lateral/Label'

In [6]:
def findcontour(threshold_img):
    contours, _ = cv2.findContours(threshold_img.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    

    print("Contours length: " + str(len(contours)))
    rect = [cv2.boundingRect(cnt) for cnt in contours]
    print("boundingRect length: " + str(len(contours)))
#     print(rect)
#     print(rect[0])
#     print(contours[0].shape)
    return rect,contours

In [20]:
def crop_spine(rect, threshold_img, org_img):
#     sub_crop_img = ''
#     sub_crop_msk = ''
    crop_imgs = []
    crop_msks = []
    i=0
    for i in range(len(rect)):
        for j,(x, y, w, h) in enumerate([rect[i]]):
            left, top, right, bottom = x, y, x+w, y+h
            sub_crop_msk = crop(threshold_img, give_border((left, top, right, bottom),30,threshold_img.shape, True))
            sub_crop_img = crop(org_img, give_border((left, top, right, bottom),30,threshold_img.shape, True))
            
#             print(msk_dir.replace(msk_dir.split('.')[-2],msk_dir.split('.')[-2]+ '_'+str(i+1)))
#             cv2.imwrite(img_dir.replace(img_dir.split('.')[-2],img_dir.split('.')[-2]+ '_'+str(i+1)), sub_crop_img)
#             cv2.imwrite(msk_dir.replace(msk_dir.split('.')[-2],msk_dir.split('.')[-2]+ '_'+str(i+1)) ,sub_crop_msk)
            crop_imgs.append(sub_crop_img)
            crop_msks.append(sub_crop_msk)
#         print(crop_imgs)
#         cv2.imwrite(img_dir,crop_imgs[i])
#         cv2.imwrite(msk_dir,crop_msks[i])
        
    return crop_imgs,crop_msks

In [8]:
#찾은 좌표 바탕으로 이미지 crop
def crop(image, coords, border=0, logging=False):
    # coords should be a tuple with (x1, y1, x2, y2)
    if logging:
        print("Shape of image: " + str(shape(image)))
    left, top, right, bottom = coords

    if isinstance(image, np.ndarray):
        return image[top: bottom, left: right]
    else:
        return image.crop((left, top, right, bottom))

In [9]:
#찾은 좌표 바탕으로 boarder 넓히기
def give_border(coords, border, shp, logging=False):
    flag = False
    coords = coords[0]-border, coords[1]-border, coords[2]+border, coords[3]+border

    if logging:
        print("Give border: " + str(coords))

    if coords[0] < 0:
        print("Left small: " + str(coords[0]))
        coords = (0, coords[1], coords[2], coords[3])
        flag = True
    if coords[1] < 0:
        print("Top small: " + str(coords[1]))
        coords = (coords[0], 0, coords[2], coords[3])
        flag = True

    if coords[2] > shp[1]:
        print("Right big: " + str(coords[2]))
        coords = (coords[0], coords[1], shp[1], coords[3])
        flag = True

    if coords[3] > shp[0]:
        print("Bottom Big: " + str(coords[3]))
        coords = (coords[0], coords[1], coords[2], shp[0])
        flag = True

    # if flag == False:
    #     print("Nothing has changed in give border")
    return coords

In [ ]:
for i in os.listdir(data_path):
    name = i.split('.')[-2]+'_label.png'
    img = os.path.join(data_path,i)
    
    image = cv2.imread(img, 0)
    mask = cv2.imread(os.path.join(mask_path,name),0)
    rect,contours = findcontour(mask)
    
#     msk_dir = os.path.join(save_msk_dir, i)
#     img_dir = os.path.join(save_img_dir, i)
    crop_imgs,crop_msks=crop_spine(rect, mask,image)
    
    for img,msk in zip(crop_imgs,crop_msks):
        fig = plt.figure()
        ax1 = fig.add_subplot(1, 2, 1)
        ax2 = fig.add_subplot(1, 2, 2)

        ax1.imshow(img,cmap='gray')
        ax2.imshow(msk,cmap='gray')
        plt.show()